<a href="https://colab.research.google.com/github/skyqi/LangChain/blob/main/langChain%E7%9A%84Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install openai
! pip install langchain


In [ ]:
import os
import getpass

api_key = os.getenv("ZHIPUAI_API_KEY")
if api_key is None:
  os.environ["ZHIPUAI_API_KEY"] = getpass.getpass()

In [ ]:
import openai,os
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAIChat
from langchain.chains import LLMChain
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_community.chat_models import ChatZhipuAI
llm = ChatZhipuAI(
    model="glm-4",
    temperature=0.5,
)

multiple_choice="""
请针对 >>> 和 <<< 中间的用户问题，选择一个合适的工具去回答它的问题。只要用A、B、C的选项字母告诉我答案。
如果你觉得都不合适，就选D。
>>>{question}<<<
我们有的工具包括：
A. 一个能够查询商品信息，为用户进行商品导购的工具
B. 一个能够查询订单信息，获得最新的订单情况的工具
C. 一个能够搜索商家的退换货政策、运费、物流时长、支付渠道、覆盖国家的工具
D. 都不合适
"""

multiple_choice_prompt = PromptTemplate(template=multiple_choice,input_variables=["question"])
choice_chain=LLMChain(llm=llm,prompt=multiple_choice_prompt, output_key="answer")

# 创建一个LLMChain来处理提示和生成响应
chain = LLMChain(llm=llm, prompt=multiple_choice_prompt)

# 定义用户问题
question = "我想买一件衣服，但是不知道哪个款式好看，你能帮我推荐一下吗？"

# 使用LLMChain来处理问题
response = chain.run(question=question)
print(response)

A. 一个能够查询商品信息，为用户进行商品导购的工具


In [ ]:
#执行第二个问题
question = "我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗？"
print(choice_chain(question))

{'question': '我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗？', 'answer': 'B. 一个能够查询订单信息，获得最新的订单情况的工具'}


In [ ]:
#执行第三个问题试试：

question = "请问你们的货，能送到三亚吗？大概需要几天？"
print(choice_chain(question))

{'question': '请问你们的货，能送到三亚吗？大概需要几天？', 'answer': 'C. 一个能够搜索商家的退换货政策、运费、物流时长、支付渠道、覆盖国家的工具\n\n这个工具能提供关于物流配送的相关信息，最符合用户询问的关于货物能否送到三亚以及大概需要几天的问题。'}


In [ ]:
#再问他都不符合的问题，
question = "今天天气怎么样？"
print(choice_chain(question))

{'question': '今天天气怎么样？', 'answer': 'D. 都不合适\n\n这个问题是关于天气的，与提供的工具功能无关，因此这些工具都不适合用来回答天气问题。'}


#定义三个函数
上面通过prompt，用LangChian既可以挑选还可以直接处理行为。
下面我们来定义三个函数
1、search_order()第一个函数是关于订单搜索，最后模拟时返回订单信息
2、recommend_product()定义了关于推荐产品的函数
3、faq()定义了电商问答的函数

In [ ]:
from langchain.agents import create_react_agent
from langchain.tools import Tool
# 模拟问关于订单
def search_order(input:str) ->str:
  return "订单状态：已发货；发货日期：2023-09-15；预计送达时间：2023-09-18"

# 模拟问关于推荐产品
def recommend_product(input:str)->str:
  return "红色连衣裙"

# 模拟问电商faq
def faq(input:str)->str:
  return "7天无理由退货"


# 创建了一个 Tool 对象的数组，把这三个函数分别封装在了三个 Tool 对象里面
# 并且定义了描述，这个 description 就是告诉 AI，这个 Tool 是干什么用的，会根据描述做出选择
tools=[
    Tool(
        name="Search Order",func=search_order,
        description="useful for when you need to answer questions about customers orders"
    ),
    Tool(
        name="Recommend Product",func=recommend_product,
        description="useful for when you need to answer questions about product recommendations"
    ),
    Tool(
        name="FAQ",func=faq,
        description="useful for when you need to answer questions about shopping policies, like return policy, shipping policy, etc."
    ),
]

# 指定使用tools，llm，agent则是zero-shot"零样本分类"，不给案例自己推理
# 而 react description，指的是根据你对于 Tool 的描述（description）进行推理（Reasoning）并采取行动（Action）
agent=initialize_agent(tools,llm,agent="zero-shot-react-description", verbose=True)


In [ ]:
#询问一下：
question = "我想买一件衣服，但是不知道哪个款式好看，你能帮我推荐一下吗？"
result=agent.run(question)
print(result)



> Entering new AgentExecutor chain...
这个问题需要推荐产品，我可以使用“推荐产品”这个工具来帮助用户选择合适的衣服款式。

Action: Recommend Product
Action Input:衣服

Observation:
Observation: 红色连衣裙
Thought:根据用户的需要，我推荐了一款红色连衣裙。但考虑到用户可能需要更多选择或者对这款推荐有疑问，我需要进一步确认。

Thought: 用户可能需要更多款式选择或者对推荐有疑问，我应该询问用户是否需要更多推荐或者对这款红色连衣裙有什么疑问。

Action: Recommend Product
Action Input: 衣服

Observation:
Observation: 红色连衣裙
Thought:由于在上一轮推荐中，我没有提供除红色连衣裙以外的其他选项，我将继续为用户推荐另一款衣服。

Thought: 这次我将为用户推荐一款不同的衣服款式，以提供更多选择。

Action: Recommend Product
Action Input: 衣服

Observation:

Observation: 红色连衣裙
Thought:抱歉，看起来在尝试推荐产品时出现了一些问题，我连续推荐了相同的红色连衣裙。为了给您提供更多样化的选择，我将再次尝试推荐其他款式的衣服。

Action: Recommend Product
Action Input: 衣服

Observation:

Observation: 红色连衣裙
Thought:由于目前推荐产品工具似乎出现了一些问题，导致我无法提供不同款式的衣服推荐，我会尝试采用其他方式来帮助您。

Thought: 既然工具目前无法提供多样化的选择，我可以根据当前流行趋势和一般审美来为用户描述几种不同的衣服款式，供用户参考。

Final Answer:以下是几种流行的衣服款式供您参考：

1. A字裙：适合大多数身材，特别能突出腰部线条，适合日常穿着。
2. 高腰裤：可以拉长腿部比例，显得身材高挑，适合正式场合。
3. 长款开衫：搭配简单上衣和牛仔裤，轻松打造休闲风格。
4. 短款夹克：可以搭配连衣裙或T恤，增加层次感。

您可以根据自己的喜好和身材特点选择合适的款式。如果您有其他问题或需要更多

首先讲讲上面打印的日志：

Action：就是根据用户的输入，选用哪个Tool，然后行动

Action Input：根据需要使用的Tool，从用户的输入里提取相关的内容，可以输入到Tool里面

Observation：就是观察通过使用 Tool 得到的一个输出结果。

Thought：就是再看一眼用户的输入，判断一下该怎么做。

Final Answer：就是 Thought 在看到 Obersavation 之后，给出的最终输出。
**加粗文字**

In [ ]:
# 问题二：
question = "我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗？"
result = agent.run(question)
print(result)



> Entering new AgentExecutor chain...
这个问题需要查询订单信息，我需要使用订单搜索工具来帮助用户解决这个问题。

Action: Search Order
Action Input: 2022ABCDE
Observation:
Observation: 订单状态：已发货；发货日期：2023-09-15；预计送达时间：2023-09-18
Thought:根据查询结果，订单已经发货，并且预计在2023年9月18日送达。现在需要告知用户订单的最新状态。

Final Answer: 您的订单（订单号：2022ABCDE）已经发货，发货日期是2023年9月15日，预计送达时间是2023年9月18日。如果到了预计送达时间您还没有收到货物，请及时联系我们，我们会进一步帮助您查询。

> Finished chain.
您的订单（订单号：2022ABCDE）已经发货，发货日期是2023年9月15日，预计送达时间是2023年9月18日。如果到了预计送达时间您还没有收到货物，请及时联系我们，我们会进一步帮助您查询。


In [ ]:
# 问题三：
question = "请问你们的货，能送到三亚吗？大概需要几天？"
result = agent.run(question)
print(result)



> Entering new AgentExecutor chain...
这个问题涉及到物流和配送，我需要调用FAQ函数来获取相关信息。

Action: FAQ
Action Input: '你们的货，能送到三亚吗？大概需要几天？'
Observation:
Observation: 7天无理由退货
Thought:对不起，我似乎调用了错误的函数。我需要使用FAQ函数来获取关于配送范围和时效的信息。

Action: FAQ
Action Input: '配送范围和时效'
Observation:
Observation: 7天无理由退货
Thought:对不起，我再次得到了错误的答案。看来我需要重新审视问题并确保我使用正确的工具来获取正确的信息。

Thought: 我需要使用FAQ工具来获取有关物流配送的具体信息。

Action: FAQ
Action Input: '是否能够配送至三亚以及所需时间'
Observation:
Observation: 7天无理由退货
Thought:我目前无法获取到关于是否能够配送至三亚以及所需时间的信息，这可能是因为工具返回的答案是关于退货政策的，而不是配送信息。由于我无法直接回答您的问题，建议您直接联系我们的客服以获取准确的配送信息。非常抱歉给您带来不便。 

Final Answer: 由于我无法获取准确的配送信息，建议您直接联系客服以确认是否能够将货物配送至三亚及所需时间。

> Finished chain.
由于我无法获取准确的配送信息，建议您直接联系客服以确认是否能够将货物配送至三亚及所需时间。


In [ ]:
#问他都不符合的问题
question = "今天天气怎么样？"
print(choice_chain(question))

{'question': '今天天气怎么样？', 'answer': 'D. 都不合适\n\n对于用户提出的“今天天气怎么样？”这个问题，以上提供的工具都不适合回答天气信息。需要使用的是天气预报工具或者搜索引擎来查询当地的天气情况。'}


In [ ]:

上面实现了，用LangChian既可以挑选还可以直接处理行为
然后我们最重要的就是tools数组，数组里放入tool函数，参数为name，func是函数，不同的功能调用不同的函数，description这个主要是类似于Prompt，
根据用户输入的是否符合描述来调用不同的函数。

创建initialize_agent()并把tools传入进来以及llm的AI模型，agent参数传递的zero-shot-react-description采用零样本分类，不给案例自己推理。

In [ ]:
from langchain.agents import initialize_agent, Tool


llm = ChatZhipuAI(
    model="glm-4",
    temperature=0.5,
)

# 模拟问关于订单
def search_order(input:str) ->str:
  return "订单状态：已发货；发货日期：2023-09-15；预计送达时间：2023-09-18"

# 模拟问关于推荐产品
def recommend_product(input:str)->str:
  return "红色连衣裙"

# 模拟问电商faq
def faq(input:str)->str:
  return "7天无理由退货"

# 创建了一个 Tool 对象的数组，把这三个函数分别封装在了三个 Tool 对象里面
# 并且定义了描述，这个 description 就是告诉 AI，这个 Tool 是干什么用的，会根据描述做出选择
tools=[
    Tool(
        name="Search Order",func=search_order,
        description="useful for when you need to answer questions about customers orders"
    ),
    Tool(
        name="Recommend Product",func=recommend_product,
        description="useful for when you need to answer questions about product recommendations"
    ),
    Tool(
        name="FAQ",func=faq,
        description="useful for when you need to answer questions about shopping policies, like return policy, shipping policy, etc."
    ),
]
# 指定使用tools，llm，agent则是zero-shot"零样本分类"，不给案例自己推理
# ChatGPT变成了一个类型为zero-shot-react-description的代理
# 而 zero-shot-react-description ，指的是根据你对于 Tool 的描述（description）进行推理（Reasoning）并采取行动（Action）
#tools: 一个包含工具的列表，每个工具都是一个 Tool 对象，它定义了工具的名称、功能和描述。
#llm: 一个大型语言模型实例，用于理解用户的指令和生成响应。
#agent: 指定要创建的代理的类型。"zero-shot-react-description" 是一种代理类型，它使用描述性语言模型来决定使用哪个工具。
# verbose: 一个布尔值，用于控制代理的输出详细程度。如果设置为 True，代理将输出更多的调试信息。

agent=initialize_agent(tools,llm,agent="zero-shot-react-description", verbose=True)




In [ ]:
question = "我想买一件衣服，但是不知道哪个款式好看，你能帮我推荐一下吗？"
result=agent.run(question)
print(result)



> Entering new AgentExecutor chain...
当然可以，我可以通过调用推荐产品的接口来帮助顾客选择合适的衣服款式。
Action: Recommend Product
Action Input:衣服
Observation:
Observation: 红色连衣裙
Thought:根据系统的推荐，我认为红色连衣裙是一个很好的选择。

Final Answer: 我推荐您选择红色连衣裙，这个款式很受欢迎，应该会非常适合您。如果您需要更多款式或其他产品的推荐，请随时告诉我。

> Finished chain.
我推荐您选择红色连衣裙，这个款式很受欢迎，应该会非常适合您。如果您需要更多款式或其他产品的推荐，请随时告诉我。


In [ ]:
#第二个问题：
question = "我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗？"
result = agent.run(question)
print(result)



> Entering new AgentExecutor chain...
这个问题需要查找订单信息，我需要使用 "Search Order" 工具来帮助我查询订单状态。

Action: Search Order
Action Input: "2022ABCDE"

Observation:

Observation: 订单状态：已发货；发货日期：2023-09-15；预计送达时间：2023-09-18
Thought:根据查询结果，订单已经发货，并且有预计的送达时间。我需要将这一信息告诉用户。

Final Answer: 您的订单（订单号：2022ABCDE）已经于2023年9月15日发货，预计将在2023年9月18日送达。请您耐心等待。如果到时候还没有收到，请您再次联系我们查询。

> Finished chain.
您的订单（订单号：2022ABCDE）已经于2023年9月15日发货，预计将在2023年9月18日送达。请您耐心等待。如果到时候还没有收到，请您再次联系我们查询。


In [ ]:
#第三个问题：
question = "请问你们的货，能送到三亚吗？大概需要几天？"
result = agent.run(question)
print(result)



> Entering new AgentExecutor chain...
这个问题涉及到物流和配送，我需要调用FAQ函数来获取相关信息。

Action: FAQ
Action Input: 三亚 配送时间
Observation:
Observation: 7天无理由退货
Thought:对不起，我需要重新调用FAQ函数来获取有关配送时间的信息。

Action: FAQ
Action Input: 三亚 配送时间
Observation:
Observation: 7天无理由退货
Thought:对不起，似乎出现了一些问题，我无法获取到关于三亚配送时间的信息。通常情况下，我们的商品配送至三亚大概需要3-5个工作日。如果您需要更准确的信息，建议直接咨询我们的客服人员，他们会为您提供详细的配送时间表。

Final Answer: 商品配送至三亚通常需要3-5个工作日，具体时间可能会有所变动，建议咨询客服获取最准确的配送信息。

> Finished chain.
商品配送至三亚通常需要3-5个工作日，具体时间可能会有所变动，建议咨询客服获取最准确的配送信息。


回复的：商品配送至三亚通常需要3-5个工作日，具体时间可能会有所变动，建议咨询客服获取最准确的配送信

在最后的案例里我们看到它在反复的重试，因为发现没有合适的，最后只能给出这样的答案，而对于这种没有确切的答案，还要反复思考重试，而且也不一定准，如果希望不要反复重试，可以设定重试的限制。

In [ ]:
#2.使用max_iterations限制重试次数
agent=initialize_agent(tools,llm,agent="zero-shot-react-description",max_iterations=2,verbose=True)
question = "请问你们的货，能送到三亚吗？大概需要几天？"
result = agent.run(question)
print(result)



> Entering new AgentExecutor chain...
这个问题涉及到物流和配送，我需要调用FAQ函数来获取相关信息。

Action: FAQ
Action Input: 三亚 配送时间
Observation:
Observation: 7天无理由退货
Thought:对不起，我需要重新调用FAQ函数来获取有关三亚配送时间的正确信息。

Action: FAQ
Action Input: 三亚 配送时间
Observation:
Observation: 7天无理由退货
Thought:

> Finished chain.
Agent stopped due to iteration limit or time limit.


下面要使用到faq小文件，暂时没有，无法往下做
https://blog.csdn.net/dfBeautifulLive/article/details/133353827?spm=1001.2014.3001.5501

跳过。。。。


In [ ]:
#优化 Prompt，让 AI 不要胡说
#!pip install ipdb
import ipdb
from langchain.agents import initialize_agent, Tool

llm = ChatZhipuAI(
    model="glm-4",
    temperature=0.5,
)

import json
import re

ORDER_1="123ABC"
ORDER_2="456EFG"

ORDER_1_DETAIL={
    "order_number":ORDER_1,
    "status":"已发货",
    "shipping_date": "2023-01-03",
    "estimated_delivered_date": "2023-01-05",
}

ORDER_2_DETAIL={
    "order_number":ORDER_2,
    "status":"未发货",
    "shipping_date": None,
    "estimated_delivered_date": None,
}

# 模拟问关于订单
def search_order(input:str) ->str:
  """useful for when you need to answer questions about customers orders"""
  # print(f"input=={input},{input}=={ORDER_1}")

  input = input.strip()
  # ipdb.set_trace()
  if re.search(ORDER_1, input):
    return json.dumps(ORDER_1_DETAIL)
  elif re.search(ORDER_2, input):
    return json.dumps(ORDER_2_DETAIL)
  else:
    return f"对不起，根据{input}没有找到您的订单"



# 模拟问关于推荐产品
def recommend_product(input:str)->str:
  return "红色连衣裙"

# 模拟问电商faq
def faq(input:str)->str:
  return "7天无理由退货"


# 创建工具列表
tools = [
    Tool(name="search_order", func=search_order, description="Search for the status of an order."),
    Tool(name="recommend_product", func=recommend_product, description="Recommend a product based on user input."),
    Tool(name="faq", func=faq, description="Answer frequently asked questions about e-commerce."),
]

# 初始化代理
agent = initialize_agent(tools=tools, llm=llm, agent="zero-shot-react-description", verbose=True)


In [ ]:
#第一问
question = "我有一张订单，订单号是989898DE，一直没有收到，能麻烦帮我查一下吗？"
answer = agent.run(question)
print(answer)



> Entering new AgentExecutor chain...
这个问题需要查询订单状态，因此我需要使用工具来帮助用户查询订单情况。

Action: search_order
Action Input: 989898DE
Observation:input==989898DE
Observation:
> <ipython-input-99-f518a5852bf0>(36)search_order()
     35   ipdb.set_trace()
---> 36   if input==ORDER_1:
     37     return json.dumps(ORDER_1_DETAIL)

ipdb> n
> <ipython-input-99-f518a5852bf0>(38)search_order()
     37     return json.dumps(ORDER_1_DETAIL)
---> 38   elif input==ORDER_2:
     39     return json.dumps(ORDER_2_DETAIL)

ipdb> q


In [ ]:
#第二问 ORDER_1="123ABC" ORDER_2="456EFG"
question = "我有一张订单，订单号是123ABC，一直没有收到，能麻烦帮我查一下吗？"
answer = agent.run(question)
print(answer)



> Entering new AgentExecutor chain...
这个问题需要查询订单状态，我可以使用提供的 `search_order` 函数来帮助用户查询订单情况。
Action: search_order
Action Input: '123ABC'
Observation:> <ipython-input-110-1aae8d4c9756>(39)search_order()
     38   ipdb.set_trace()
---> 39   if re.search(ORDER_1, input):
     40     return json.dumps(ORDER_1_DETAIL)

ipdb> n
> <ipython-input-110-1aae8d4c9756>(40)search_order()
     39   if re.search(ORDER_1, input):
---> 40     return json.dumps(ORDER_1_DETAIL)
     41   elif re.search(ORDER_2, input):

ipdb> n
--Return--
'{"order_numb..."2023-01-05"}'
> <ipython-input-110-1aae8d4c9756>(40)search_order()
     39   if re.search(ORDER_1, input):
---> 40     return json.dumps(ORDER_1_DETAIL)
     41   elif re.search(ORDER_2, input):

ipdb> c

Observation: {"order_number": "123ABC", "status": "\u5df2\u53d1\u8d27", "shipping_date": "2023-01-03", "estimated_delivered_date": "2023-01-05"}
Thought:根据查询结果，订单已经发货，发货日期是2023年1月3日，预计送达日期是2023年1月5日。由于今天已经是1月5日，如果用户还没有收到货物，可能是遇到了运输延迟。
Thought: 我需要告诉用户订单的